In [41]:
%pip install mlflow==1.30
%pip install boto3
%pip install cloudpickle==2.2.1

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install psycopg2-binary

In [ ]:
%pip install feast[cassandra]==0.26

In [2]:
import mlflow

In [3]:
mlflow.__version__

'1.30.0'

In [38]:
from feast import FeatureStore
store = FeatureStore()

In [39]:
from datetime import datetime, timedelta
store.materialize(
    feature_views=["credit_history"],
    start_date=datetime.utcnow() - timedelta(days=1000), end_date=datetime.utcnow() - timedelta(minutes=10)
)

Materializing 1 feature views from 2020-12-30 12:28:10+00:00 to 2023-09-26 12:18:10+00:00 into the cassandra online store.

credit_history:


 25%|██████████████▎                                           | 7073/28633 [01:24<04:17, 83.75it/s]


KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta
store.materialize(
    feature_views=["zipcode_features"],
    start_date=datetime.utcnow() - timedelta(days=3650), end_date=datetime.utcnow() - timedelta(minutes=10)
)

In [ ]:
store.materialize(
    feature_views=["client_features"],
    start_date=datetime.utcnow() - timedelta(days=3650), end_date=datetime.utcnow() - timedelta(minutes=10)
)

In [40]:

from sklearn import tree
from sklearn import ensemble
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from mlflow.models.signature import infer_signature
from mlflow.models import MetricThreshold
import pandas as pd
import dill
import sklearn
import json

from feast import FeatureStore

store = FeatureStore(repo_path=".")
dataset_name = 'loan_approval_baseline'
training_df = store.get_saved_dataset(dataset_name).to_df()

# cfg = {"type": "feast_dataset", "fs_path": ".", "dataset_path": "loan_approval_baseline", "categorical_features": ["person_home_ownership", "loan_intent", "state"], "cols_to_drop": ["created_timestamp", "event_timestamp", "loan_id", "zipcode", "dob_ssn", "city", "location_type"]}

# import json
# with open("cfg.json", "w") as f:
#     json.dump(cfg, f)

encoder = OrdinalEncoder()
categorical_features = [
    "person_home_ownership",
    "loan_intent",
    "state",
]
cols_to_drop = ['created_timestamp', 'event_timestamp', 'loan_id', 'zipcode', 'dob_ssn', 'city', 'location_type']

encoder.fit(training_df[categorical_features])
transform_training_df=training_df.copy()
transform_training_df[categorical_features] = encoder.transform(
    training_df[categorical_features]
)

target = "loan_status"
train_X = transform_training_df[
    transform_training_df.columns.drop(target)
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("person_income")
    .drop("zipcode")
    .drop("dob_ssn")
    .drop("city")
    .drop("location_type")
]
train_X = train_X.reindex(sorted(train_X.columns), axis=1)
train_Y = transform_training_df.loc[:, target]

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.10)

n_estimators = 100
classifier = ensemble.GradientBoostingClassifier(n_estimators=n_estimators)
classifier.fit(x_train, y_train)
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_true=y_test, y_pred = predictions)

def eval_metrics(actual, pred):
    precision = precision_score(actual, pred)
    recall = recall_score(actual, pred)
    f1 = f1_score(actual, pred)
    return precision, recall, f1

(precision, recall, f1) = eval_metrics(y_test, predictions)
print(precision)
print(recall)
print(f1)

import joblib

model_name = "model.bin"
encoder_name = "encoder.bin"

joblib.dump(classifier, model_name)
joblib.dump(encoder, encoder_name)

artifacts = {
    "model": model_name,
    "encoder": encoder_name,
}

data_cfg_name = "data_cfg.json"

data_cfg = {
    'type': 'feast_dataset',
    'fs_path': ".",
    'dataset_path': dataset_name,
    'categorical_features': categorical_features,
    'cols_to_drop': cols_to_drop,
}

with open(data_cfg_name, 'w') as fp:
    json.dump(data_cfg, fp)

target_column = 'loan_status'
eval_data = x_test
eval_data[target_column] = y_test

eval_train_df = training_df.copy()
eval_train_df=eval_train_df[
    eval_train_df.columns
    .drop("event_timestamp")
    .drop("created_timestamp")
    .drop("loan_id")
    .drop("person_income")
    .drop("city")
    .drop("location_type")
]

signature = infer_signature(x_test, predictions)

wrapped_signature = infer_signature

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import logging

class ModelWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import joblib
        self.encoder = joblib.load(context.artifacts["encoder"])            
        self.model = joblib.load(context.artifacts["model"]) 
        self.categorical_features = [
            "person_home_ownership",
            "loan_intent",
            "state",
        ]

    def predict(self, context, inp):

        import pandas as pd

        logger = logging.getLogger(__name__)

        if isinstance(inp, dict):
            features_df = pd.DataFrame.from_dict(inp)
        elif isinstance(inp, pd.DataFrame):
            features_df = inp

        logger.debug("Enriching features DataFrame: %s", features_df)

        # Apply ordinal encoding to categorical features
        features_df[self.categorical_features] = self.encoder.transform(
            features_df[self.categorical_features]
        )

        logger.debug("features DataFrame after encoging: %s", features_df)

        # Sort columns
        features_df = features_df.reindex(sorted(features_df.columns), axis=1)

        # Drop unnecessary columns
        df_for_predict = features_df.copy()
        if "city" in df_for_predict.columns:
            df_for_predict = df_for_predict[df_for_predict.columns.drop("city").drop("location_type")]

        if "zipcode" in df_for_predict.columns:
            df_for_predict = df_for_predict[df_for_predict.columns.drop("zipcode").drop("dob_ssn")]

        logger.debug("features DataFrame before predict: %s", df_for_predict)

        # Make prediction
#         df_for_predict["prediction"] = self.model.predict(df_for_predict)
        
        result = self.model.predict(df_for_predict)

#         result = pd.merge(features_df, df_for_predict, left_index=True, right_index=True)[["zipcode", "dob_ssn", "prediction"]]

#         logger.debug("result of predict: %s", result)

        # return result of credit scoring
#         return result.to_numpy()
        return result
#                 return df_for_predict.to_numpy()

conda_env={
    'channels': ['defaults'],
    'dependencies': [
      'python=3.8.10',
      'pip',
      {
        'pip': [
          'mlflow=={}'.format(mlflow.__version__),
          'numpy==1.23.4',
          'scikit-learn=={}'.format(sklearn.__version__),
          'joblib=={}'.format(joblib.__version__),
          'dill=={}'.format(dill.__version__),
#           'numpy==1.19',
        ],
      },
    ],
    'name': 'demo_env'
}

thresholds = {
#     "mean_ks_test": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "min_ks_test": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "mean_chi_square": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
#     "min_chi_square": MetricThreshold(
#         threshold=0.05,
#         higher_is_better=True,
#     ),
    "accuracy_score": MetricThreshold(
        threshold=0.05,
        higher_is_better=True,
    ),
}

mlflow.set_experiment("loans_demo_dev2")

with mlflow.start_run(run_name='train baseline model') as current_run:

    mlflow.log_param("n_estimators", n_estimators)

    mlflow.set_tag('base_dataset', dataset_name)

    model_without_wrapper = mlflow.sklearn.log_model(
        classifier, "model",  
        conda_env=conda_env, 
        signature=signature
    )
    
#     mlflow.log_artifact(data_cfg_name)

    result = mlflow.evaluate(
        model_without_wrapper.model_uri,
        eval_data,
        targets=target_column,
        model_type="classifier",
        evaluators=["default"],
        validation_thresholds=thresholds,
    )

    with mlflow.start_run(run_name='train baseline model', nested=True) as current_run:
        model_info = mlflow.pyfunc.log_model(
            artifact_path="model", 
            python_model=ModelWrapper(),
            artifacts=artifacts,
            conda_env=conda_env,
#                     signature=signature
        )
        
        mlflow.log_artifact(data_cfg_name)

        result = mlflow.evaluate(
            model_info.model_uri,
            eval_train_df,
            targets=target_column,
            model_type="classifier",
            evaluators=["default"],
            validation_thresholds=thresholds,
        )

    run_id = current_run.info.run_id
    print(run_id)


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1235: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


0.8533333333333334
0.6124401913875598
0.713091922005571


<ipython-input-40-8dbdac5da7da>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_data[target_column] = y_test
/opt/conda/lib/python3.8/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:

4c05c6362245402189d9f72c90b9ca13


<Figure size 1050x700 with 0 Axes>

In [ ]:
mlflow.pyfunc.get_model_dependencies(model_uri)

In [ ]:
%pip install feast[cassandra]==0.26

In [ ]:
%pip install psycopg2-binary

In [11]:
import pandas as pd

In [12]:
pd.read_parquet("loan_approval_2022q3.parquet").dtypes

loan_id                    int64
dob_ssn                   object
zipcode                    int64
person_age                 int64
person_income              int64
person_home_ownership     object
person_emp_length        float64
loan_intent               object
loan_amnt                  int64
loan_int_rate            float64
loan_status                int64
event_timestamp           object
created_timestamp         object
city                      object
state                     object
location_type             object
tax_returns_filed          int64
population                 int64
total_wages                int64
credit_card_due            int64
mortgage_due               int64
student_loan_due           int64
vehicle_loan_due           int64
hard_pulls                 int64
missed_payments_2y         int64
missed_payments_1y         int64
missed_payments_6m         int64
bankruptcies               int64
dtype: object

In [13]:
loans = pd.read_parquet("loan_table.parquet")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
loans.columns

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status', 'event_timestamp',
       'created_timestamp'],
      dtype='object')

In [15]:
q3 = pd.read_parquet("loan_approval_baseline.parquet")[loans.columns]

In [16]:
loan_features = store.get_feature_service('loan_features')

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
merged

NameError: name 'merged' is not defined

In [18]:
training_df = training_features.to_df()

training_df

NameError: name 'training_features' is not defined

In [19]:
q3

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,24648,19490816_6592,44287,23,120000,MORTGAGE,5.0,VENTURE,6000,9.99,0,2021-02-20 03:47:05.196000+00:00,2021-02-20 03:47:05.196000+00:00
1,28499,19831120_6431,33620,27,45000,MORTGAGE,11.0,PERSONAL,3450,11.99,0,2021-01-02 01:46:18.180000+00:00,2021-01-02 01:46:18.180000+00:00
2,36222,19911026_9586,92110,36,39520,OWN,14.0,VENTURE,5000,5.79,0,2020-09-25 15:19:18.262000+00:00,2020-09-25 15:19:18.262000+00:00
3,15836,19630507_1213,13865,24,57000,RENT,0.0,VENTURE,6000,7.90,0,2021-06-12 11:21:27.493000+00:00,2021-06-12 11:21:27.493000+00:00
4,14019,19671129_3523,97523,23,124000,RENT,2.0,PERSONAL,10000,14.27,0,2021-07-05 15:10:27.256000+00:00,2021-07-05 15:10:27.256000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,23050,19720313_3006,71749,23,116004,MORTGAGE,1.0,PERSONAL,14000,12.99,0,2021-03-12 12:36:35.004000+00:00,2021-03-12 12:36:35.004000+00:00
9565,20561,19570213_9898,33417,24,78500,MORTGAGE,8.0,VENTURE,6000,7.49,0,2021-04-13 05:59:23.122000+00:00,2021-04-13 05:59:23.122000+00:00
9566,37356,19600118_8531,60108,37,77000,MORTGAGE,2.0,VENTURE,15000,12.29,0,2020-09-11 04:26:00.413000+00:00,2020-09-11 04:26:00.413000+00:00
9567,25458,19981011_8953,20601,23,41000,RENT,7.0,EDUCATION,4700,7.90,0,2021-02-09 20:00:26.732000+00:00,2021-02-09 20:00:26.732000+00:00


In [20]:
q3["loan_id"].values

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([24648, 28499, 36222, ..., 37356, 25458, 24530])

In [21]:
q3.sort_values('loan_id')

,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
4347,10000,19530219_5179,76104,22,59000,RENT,123.0,PERSONAL,35000,16.02,1,2021-08-25 20:34:41.361000+00:00,2021-08-25 20:34:41.361000+00:00
643,10011,19880808_5747,55307,21,10000,OWN,2.0,HOMEIMPROVEMENT,4500,8.63,1,2021-08-25 17:12:47.802000+00:00,2021-08-25 17:12:47.802000+00:00
6804,10013,19561221_5856,1867,26,108160,RENT,4.0,EDUCATION,35000,18.39,1,2021-08-25 16:36:05.336000+00:00,2021-08-25 16:36:05.336000+00:00
1022,10018,19751208_7623,20692,23,113000,RENT,8.0,DEBTCONSOLIDATION,35000,18.25,1,2021-08-25 15:04:19.173000+00:00,2021-08-25 15:04:19.173000+00:00
2177,10019,19710322_7014,48827,24,10800,MORTGAGE,8.0,EDUCATION,1750,10.99,1,2021-08-25 14:45:57.940000+00:00,2021-08-25 14:45:57.940000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,38360,19851025_1753,95448,39,40000,OWN,4.0,VENTURE,1000,10.37,0,2020-08-29 09:18:42.811000+00:00,2020-08-29 09:18:42.811000+00:00
5392,38361,19981215_8450,81657,39,80000,MORTGAGE,18.0,MEDICAL,5600,13.43,0,2020-08-29 09:00:21.578000+00:00,2020-08-29 09:00:21.578000+00:00
3546,38365,19630319_8917,62356,37,32000,RENT,4.0,MEDICAL,10000,13.79,1,2020-08-29 07:46:56.648000+00:00,2020-08-29 07:46:56.648000+00:00
5509,38369,19490626_3291,70083,40,43000,MORTGAGE,18.0,PERSONAL,2500,12.84,0,2020-08-29 06:33:31.717000+00:00,2020-08-29 06:33:31.717000+00:00


In [22]:
test = loans[loans["loan_id"].isin(q3["loan_id"].values)]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
loan_times = loans[["loan_id", "event_timestamp", "created_timestamp"]]

In [24]:
loan_times

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,loan_id,event_timestamp,created_timestamp
0,10000,2021-08-25 20:34:41.361000+00:00,2021-08-25 20:34:41.361000+00:00
1,10001,2021-08-25 20:16:20.128000+00:00,2021-08-25 20:16:20.128000+00:00
2,10002,2021-08-25 19:57:58.896000+00:00,2021-08-25 19:57:58.896000+00:00
3,10003,2021-08-25 19:39:37.663000+00:00,2021-08-25 19:39:37.663000+00:00
4,10004,2021-08-25 19:21:16.430000+00:00,2021-08-25 19:21:16.430000+00:00
...,...,...,...
28633,38633,2020-08-25 21:48:06.292000+00:00,2020-08-25 21:48:06.292000+00:00
28634,38634,2020-08-25 21:29:45.059000+00:00,2020-08-25 21:29:45.059000+00:00
28635,38635,2020-08-25 21:11:23.826000+00:00,2020-08-25 21:11:23.826000+00:00
28636,38636,2020-08-25 20:53:02.594000+00:00,2020-08-25 20:53:02.594000+00:00


In [25]:
q3_without_time = q3[['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status']]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
pd.concat([q3_without_time.set_index('loan_id'),loan_times.set_index('loan_id')], axis=1, join='inner').reset_index()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,loan_id,dob_ssn,zipcode,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,event_timestamp,created_timestamp
0,24648,19490816_6592,44287,23,120000,MORTGAGE,5.0,VENTURE,6000,9.99,0,2021-02-20 03:47:05.196000+00:00,2021-02-20 03:47:05.196000+00:00
1,28499,19831120_6431,33620,27,45000,MORTGAGE,11.0,PERSONAL,3450,11.99,0,2021-01-02 01:46:18.180000+00:00,2021-01-02 01:46:18.180000+00:00
2,36222,19911026_9586,92110,36,39520,OWN,14.0,VENTURE,5000,5.79,0,2020-09-25 15:19:18.262000+00:00,2020-09-25 15:19:18.262000+00:00
3,15836,19630507_1213,13865,24,57000,RENT,0.0,VENTURE,6000,7.90,0,2021-06-12 11:21:27.493000+00:00,2021-06-12 11:21:27.493000+00:00
4,14019,19671129_3523,97523,23,124000,RENT,2.0,PERSONAL,10000,14.27,0,2021-07-05 15:10:27.256000+00:00,2021-07-05 15:10:27.256000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,23050,19720313_3006,71749,23,116004,MORTGAGE,1.0,PERSONAL,14000,12.99,0,2021-03-12 12:36:35.004000+00:00,2021-03-12 12:36:35.004000+00:00
9565,20561,19570213_9898,33417,24,78500,MORTGAGE,8.0,VENTURE,6000,7.49,0,2021-04-13 05:59:23.122000+00:00,2021-04-13 05:59:23.122000+00:00
9566,37356,19600118_8531,60108,37,77000,MORTGAGE,2.0,VENTURE,15000,12.29,0,2020-09-11 04:26:00.413000+00:00,2020-09-11 04:26:00.413000+00:00
9567,25458,19981011_8953,20601,23,41000,RENT,7.0,EDUCATION,4700,7.90,0,2021-02-09 20:00:26.732000+00:00,2021-02-09 20:00:26.732000+00:00


In [27]:
merged = q3_without_time.merge(loan_times, on="loan_id", how="left")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source import (
    PostgreSQLSource, SavedDatasetPostgreSQLStorage
)

training_features = store.get_historical_features(
    entity_df=merged, 
    features=loan_features
)

dataset = store.create_saved_dataset(
    from_=training_features,
    name=f'loan_approval_baseline',
    storage=SavedDatasetPostgreSQLStorage(table_ref=f'loan_approval_baseline')
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


DuplicateTable: relation "loan_approval_baseline" already exists


In [ ]:
merged

In [29]:
import mlflow
logged_model = 'runs:/7c6fd1832f0b48cbb7d00e2b8141742d/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.

# loaded_model.predict(training_df.drop(["person_income", "loan_status"]))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2023/09/26 11:19:20 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 1.22.3, required: numpy==1.23.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [30]:
training_df.columns

Index(['loan_int_rate', 'loan_intent', 'person_income', 'loan_status',
       'person_age', 'dob_ssn', 'zipcode', 'loan_id', 'person_emp_length',
       'loan_amnt', 'created_timestamp', 'person_home_ownership', 'city',
       'state', 'location_type', 'tax_returns_filed', 'population',
       'total_wages', 'credit_card_due', 'mortgage_due', 'student_loan_due',
       'vehicle_loan_due', 'hard_pulls', 'missed_payments_2y',
       'missed_payments_1y', 'missed_payments_6m', 'bankruptcies',
       'incomeamount12m', 'event_timestamp'],
      dtype='object')

In [31]:
training_df.drop(["person_income", "loan_status"], axis=1).dtypes

loan_int_rate                   float64
loan_intent                      object
person_age                        int64
dob_ssn                          object
zipcode                           int64
loan_id                           int64
person_emp_length               float64
loan_amnt                         int64
created_timestamp        datetime64[ns]
person_home_ownership            object
city                             object
state                            object
location_type                    object
tax_returns_filed                 int64
population                        int64
total_wages                       int64
credit_card_due                   int64
mortgage_due                      int64
student_loan_due                  int64
vehicle_loan_due                  int64
hard_pulls                        int64
missed_payments_2y                int64
missed_payments_1y                int64
missed_payments_6m                int64
bankruptcies                      int64


In [33]:
loaded_model.predict(training_df[["zipcode","dob_ssn","tax_returns_filed","total_wages","state","population","student_loan_due","credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies","missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m","person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"]])

<ipython-input-9-8dbdac5da7da>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df[self.categorical_features] = self.encoder.transform(


array([0, 0, 0, ..., 0, 0, 0])

In [35]:
dataset_name = 'loan_approval_2022q3_m1'
training_df2 = store.get_saved_dataset(dataset_name).to_df()

In [36]:
training_df2[["zipcode","dob_ssn","tax_returns_filed","location_type","city","total_wages","state","population","student_loan_due","credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies","missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m","person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"]]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,zipcode,dob_ssn,tax_returns_filed,location_type,city,total_wages,state,population,student_loan_due,credit_card_due,...,hard_pulls,mortgage_due,missed_payments_2y,incomeamount12m,person_age,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate
0,64730,19710923_6795,3134,PRIMARY,BUTLER,85617594,MO,5629,4764,2308,...,5,1317248,4,204756.0,23,RENT,1.0,HOMEIMPROVEMENT,6250,16.82
1,23833,19540514_5083,991,PRIMARY,CHURCH ROAD,39171958,VA,1846,18115,5366,...,2,1014189,0,236280.0,22,RENT,0.0,VENTURE,2500,9.91
2,59935,19500909_5696,1475,PRIMARY,TROY,32636522,MT,2615,22988,9459,...,10,1212162,2,234888.0,23,RENT,0.0,PERSONAL,12000,10.37
3,21117,19940306_9908,25767,PRIMARY,OWINGS MILLS,1520024734,MD,43195,31004,8379,...,10,470349,0,230664.0,21,RENT,3.0,EDUCATION,4800,11.12
4,56345,19740620_7017,6979,PRIMARY,LITTLE FALLS,214528388,MN,12016,21548,2210,...,0,1892410,0,239304.0,22,MORTGAGE,6.0,VENTURE,2400,5.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1469,19650213_4791,3457,PRIMARY,TOWNSEND,175478972,MA,6266,7451,3109,...,6,624769,0,165612.0,32,MORTGAGE,16.0,DEBTCONSOLIDATION,3000,7.51
8210,83209,19620320_5684,296,PRIMARY,POCATELLO,4971841,ID,484,32363,658,...,4,1024135,7,146484.0,24,RENT,2.0,EDUCATION,18000,13.48
8211,68760,20010514_4088,498,PRIMARY,NIOBRARA,9981758,NE,895,26776,8410,...,3,471210,2,239724.0,23,OWN,8.0,VENTURE,6000,12.87
8212,44656,19570622_8275,1318,PRIMARY,MINERAL CITY,40440220,OH,2337,4479,338,...,6,243918,7,162600.0,24,MORTGAGE,5.0,DEBTCONSOLIDATION,14000,7.51


In [ ]:
loans.columns

In [ ]:
"person_age","person_home_ownership","person_emp_length","loan_intent","loan_amnt","loan_int_rate"

In [ ]:
training_df

In [ ]:
training_df3 = 

In [ ]:
categorical_features = [
    "person_home_ownership",
    "loan_intent",
    "city",
    "state",
    "location_type",
]
numeric_features = [
    "zipcode","dob_ssn","tax_returns_filed","total_wages","population","student_loan_due",
    "credit_card_due","missed_payments_1y","vehicle_loan_due","bankruptcies",
    "missed_payments_6m","hard_pulls","mortgage_due","missed_payments_2y","incomeamount12m",
    "person_age","person_emp_length","loan_amnt","loan_int_rate"
]
target = "loan_status"

In [ ]:
categorical_features + numeric_features + [target]

In [ ]:
training_df[categorical_features + numeric_features + [target]]

In [ ]:
from mlflow import MlflowClient
client = MlflowClient()

In [ ]:
name = "loan_approval"

In [ ]:
client.get_latest_versions(name)

In [ ]:
client.get_model_version(name, 4)

In [ ]:
client.get_model_version_download_uri(name, 4)

In [ ]:
loaded_model.metadata.run_id

In [ ]:
model_info.model_uri

In [ ]:
loaded_model

In [ ]:
client.get_registered_model(name)

In [ ]:
rm = client.get_registered_model(name)

In [ ]:
rm.latest_versions[-1].run_id